# ABS Quarterly Dwelling Stock 6432

## Python set-up

In [1]:
# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

# local imports
import abs_data_capture as adc
import plotting as p

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999

# display charts within this notebook
SHOW = False

## Get data from ABS

In [2]:
landing_page_ = adc.AbsLandingPage(
    theme="economy",
    parent_topic="price-indexes-and-inflation",
    topic="total-value-dwellings",
)
abs_dict = adc.get_abs_data(landing_page_)
source, CHART_DIR, Cat_ID, meta = adc.get_fs_constants(abs_dict, landing_page_)
RECENT, plot_times, plot_tags = adc.get_plot_constants(meta)

# plotting
p.clear_chart_dir(CHART_DIR)
p.set_chart_dir(CHART_DIR)
plt.style.use("fivethirtyeight")

A little unexpected: We need to fake up a zip file
Retrieving data from cache.
Retrieving data from cache.
Extracting DataFrames from the zip-file.


In [3]:
print(abs_dict.keys())

dict_keys(['1', '2', 'META_DATA'])


In [4]:
print(abs_dict["1"].tail().index)

PeriodIndex(['2022Q4', '2023Q1', '2023Q2', '2023Q3', '2023Q4'], dtype='period[Q-DEC]', name='Series ID')


## Plot the data

In [5]:
def headline() -> None:
    """Headline charts"""

    plotable = [
        "Value of dwelling stock; Owned by All Sectors ;  Australia ;",
        "Mean price of residential dwellings ;  Australia ;",
        "Number of residential dwellings ;  Australia ;",
    ]
    data = abs_dict["1"]
    for item in plotable:
        search = {item: adc.metacol.did}
        series_id, units = adc.find_id(meta, search)
        series, units = p.recalibrate_series(data[series_id], units, verbose=True)

        ax = series.plot()
        p.finalise_plot(
            ax,
            title=item.split(";", maxsplit=1)[0],
            ylabel=units,
            rfooter=source,
            lfooter="Australia. ",
            show=SHOW,
        )

In [6]:
headline()

recalibrate(): No adjustments needed


## Calculate population per dwelling

In [7]:
def get_population() -> tuple[pd.Series, str]:
    """Get latest population estimates from national accounts."""

    landing_page = adc.AbsLandingPage(
        theme="economy",
        parent_topic="national-accounts",
        topic="australian-national-accounts-national-income-expenditure-and-product",
    )
    gdp_dict = adc.get_abs_data(landing_page)
    _, _, cat_id, gdp_meta = adc.get_fs_constants(gdp_dict, landing_page)

    gdp_did = "Gross domestic product"
    gdp_pc_did = "GDP per capita"
    table = "1"
    data = pd.DataFrame()
    for item in gdp_did, gdp_pc_did:
        search = {
            item: adc.metacol.did,
            "Chain volume measures": adc.metacol.did,
            "$": adc.metacol.unit,
            "Original": adc.metacol.stype,
            table: adc.metacol.table,
        }
        series_id, _unit = adc.find_id(gdp_meta, search, verbose=False)
        data[item] = gdp_dict[table][series_id]
    population = data[data.columns[0]] / data[data.columns[1]] * 1_000_000

    return population, cat_id

In [8]:
def pop_per_dwelling() -> None:
    """Plot population per dwelling."""

    population, cat_gdp = get_population()

    search = {"Number of residential dwellings ;  Australia ;": adc.metacol.did}
    series_id, units = adc.find_id(meta, search)
    assert units == "Thousands"
    dwellings = abs_dict["1"][series_id] * 1_000
    pop_per_dwellings = (population / dwellings).dropna()

    ax = pop_per_dwellings.plot()
    p.finalise_plot(
        ax,
        title="Population per dwelling",
        ylabel="Persons",
        rfooter=f"{source} {cat_gdp}",
        lfooter="Australia. ",
        show=SHOW,
    )

In [9]:
pop_per_dwelling()

Retrieving data from cache.
Extracting DataFrames from the zip-file.


## Finished

In [10]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Tue Mar 12 2024 13:22:06

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.22.2

pandas    : 2.2.1
matplotlib: 3.8.3

Watermark: 2.4.3



In [11]:
print("Finished")

Finished
